In [ ]:
import numpy as np
import pandas as pd

def explode(df_, explode_cols, sep = None, fill_value=float('nan'), preserve_index=False):
    '''Split entries with multiple values separated by a separator into multiple rows (or entries in lists)
    
    https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows
    
    df_: pd.DataFrame
    explode_cols: list
        columns to split
    sep: separator
        if None, already in list format
    fill_value: 
        if empty entry, what to fill it with
    preserve_index: bool
        keep original index values for each new row
    
    '''
    res = None
    for col in explode_cols:
        if res is None:
            df = df_.copy()
        else:
            df = res.copy()
        
        df[col]=df[col].str.split(sep)

        idx_cols = df.columns.difference([col])
        # calculate lengths of lists
        lens = df[col].apply(lambda x: len(x))

        idx = np.repeat(df.index.values, lens)
        # create "exploded" DF
        res = (pd.DataFrame({
                    col_:np.repeat(df[col_].values, lens)
                    for col_ in idx_cols},
                    index=idx)
                 .assign(**{col_:np.concatenate(df.loc[lens>0, col_].values)
                                for col_ in [col]}))
        # append those rows that have empty lists
        if (lens == 0).any():
            # at least one list in cells is empty
            res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                      .fillna(fill_value))
        # revert the original index order
        res = res.sort_index()
        # reset index if requested
        if not preserve_index:        
            res.reset_index(drop=True, inplace = True)
    return res